In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mpf
import os
from pandas import to_datetime

In [ ]:
data_test = pd.read_excel(r'C:\Users\USER\Paper\Test_2023.xlsx')

In [ ]:
df = data_test.rename(columns={'年月日':'Date',
                              '開盤價(元)':'Open',
                              '最高價(元)':'High',
                              '最低價(元)':'Low',
                              '收盤價(元)':'Close',})

In [ ]:
df_drop = df.drop_duplicates('代號',keep='first').reset_index(drop=True)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
figure_paths = []
open_price = []
high_price = []
low_price = []
close_price = []
index_list = []
test_scale_min = 1500
test_scale_max = test_scale_min + 300
width_in = 170 / 100
height_in = 340 / 100

index_df = df[df['代號'] == 'Y9999'].reset_index(drop=True)
for code in df_drop['代號'].values[test_scale_min:test_scale_max]:
    draw_df = df[df['代號'] == code].reset_index(drop=True) 

    for i in range(len(draw_df)-150):
        
        stock = draw_df[i:i+150]
        stock = stock.iloc[:,2:].set_index('Date')
        idx = index_df[i:i+150]
        idx = idx.iloc[:,2:].set_index('Date')
        
        if not os.path.exists(code):
            os.makedirs(code)
            
        fig, ax1 = plt.subplots(figsize=(width_in, height_in), tight_layout=True) 
        mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
        s  = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)
        mc_idx = mpf.make_marketcolors(up='b', down='purple', inherit=True)
        s_idx  = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc_idx)
        
        fig_path = os.path.join(code, f"{code}_{i}.png")
        
        mpf.plot(stock,
                type='candle',
                volume=False,
                ax=ax1,
                style=s,
                tight_layout=True)
        
        ax2 = ax1.twinx()        
        mpf.plot(idx,
                type='candle',
                volume=False,
                ax=ax2,
                style=s_idx,
                tight_layout=True)
        ax1.axis('off')
        ax2.axis('off')        
        plt.savefig(fig_path)
        
        figure_paths.append(fig_path)
        
        open_price.append(stock['Open'].iloc[-1])
        high_price.append(stock['High'].iloc[-1])
        low_price.append(stock['Low'].iloc[-1])
        close_price.append(stock['Close'].iloc[-1])
        index_list.append(stock.index[-1])
        
figure_data = pd.DataFrame({'File Paths': figure_paths,
                            'Open':open_price,
                            'High':high_price,
                            'Low':low_price,
                            'Close':close_price,
                            'Date': index_list})
figure_data['code'] = figure_data.iloc[:, 0].str.split("\\").str[0] 
figure_data.to_csv('Test_data' + '_' + str(test_scale_min) + '_' + str(test_scale_max) +'.csv')